In [82]:
# Import necessary packages
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from parse import preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [83]:
df = preprocess("rawfile_blood.csv")


####################################################################
Number of Rows of Dataframe:
1123
Number of Columns of Dataframe:
59

####################################################################
Threshold for number of NULLs in a column: 0.1095
Number of Columns before Parsing for Too Many NULLs in a column:
59
Number of Columns after Parsing for Too Many NULLs in a column:
51

Columns Removed:
B1_b5
B4_a1
B4_a3
B4_a4
B4_a6
B4_b1
B4_b3
B5_a1

####################################################################
Number of Rows before Parsing NULLs in data:
1123
Number of Rows after Parsing NULLs in data:
1007

####################################################################
Number of Columns after dropping A1_2, B1_b4, B2_c3, B4_b2 for inconsistent data types:
47


In [84]:
# Initialise counters for each condition
frail = 0
frail_mci = 0
mci = 0
prefrail_mci = 0
prefrail = 0
robust = 0

# Count rows of data for each condition
for i in range(0, len(df)):
	if df.at[i, 'condition'] == 'frail':
		frail += 1
	elif df.at[i, 'condition'] == 'frail_mci':
		frail_mci += 1
	elif df.at[i, 'condition'] == 'mci':
		mci += 1
	elif df.at[i, 'condition'] == 'prefrail_mci':
		prefrail_mci += 1
	elif df.at[i, 'condition'] == 'prefrail':
		prefrail += 1
	elif df.at[i, 'condition'] == 'robust':
		robust += 1
        
# Display number of rows (frequency) for each condition (label)
print("\n####################################################################")
print("Labels with frequencies:")
print("Frail:", frail)
print("Frail + MCI:", frail_mci)
print("MCI:", mci)
print("Prefrail + MCI:", prefrail_mci)
print("Prefrail:", prefrail)
print("Robust:", robust)


####################################################################
Labels with frequencies:
Frail: 7
Frail + MCI: 76
MCI: 133
Prefrail + MCI: 231
Prefrail: 221
Robust: 339


In [85]:
# Specify features and labels
y = df['condition']
x = df.drop(['mtag', 'condition'], axis=1)

# Display features and labels
print(x, y)

# Display shape of features and labels
print("\nShape of Features:")
print(x.shape)
print("\nShape of Labels:")
print(y.shape)

# Standardizing the features
x = StandardScaler().fit_transform(x)

y = np.array(y)

      A1_1  A2_1  A3_1  B1_a  B1_a1  B1_a2  B1_a3  B1_a4  B1_a5  B1_a6  ...  \
0      196    24  46.5   121   3.93   0.37     95     31    324   13.3  ...   
1      200    23  55.6   142   4.82   0.42     87     30    346   12.8  ...   
2      441    20  76.8   105   4.54   0.41     90     30    330   14.0  ...   
3      265    16  47.2   122   4.53   0.39     86     27    313   14.9  ...   
4      425    14  31.3   124   4.44   0.38     85     28    329   12.6  ...   
...    ...   ...   ...   ...    ...    ...    ...    ...    ...    ...  ...   
1002   220    19  67.5   138   4.66   0.42     91     30    325   14.1  ...   
1003   334    18  51.0   139   4.63   0.42     91     30    330   15.6  ...   
1004   418    17  61.0   122   4.18   0.38     90     29    324   13.5  ...   
1005   393    18  43.1   136   4.57   0.43     94     30    316   12.5  ...   
1006   371    24  55.9   127   4.41   0.40     90     29    320   13.8  ...   

      B2_d6  B2_d7  B2_d8  B2_d9   B3  B4_a2  B4_a5

In [86]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df[['condition']]], axis = 1)

y = finalDf['condition']
x = finalDf.drop(['condition'], axis=1)

In [87]:
# Conduct train-test split on dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Display x_train, x_test, y_train, y_test
print("\nX Train:")
print(x_train)
print("\nX Test:")
print(x_test)
print("\nY Train:")
print(y_train)
print("\nY Test:")
print(y_test)

# Display shape of train and test sets
print("\nShape of X Train:")
print(x_train.shape)
print("\nShape of X Test:")
print(x_test.shape)
print("\nShape of Y Train:")
print(y_train.shape)
print("\nShape of Y Test:")
print(y_test.shape)


X Train:
     principal component 1  principal component 2
559              -1.299210               0.064637
966               0.792738              -1.633264
874               4.435945              -1.023632
6                 4.491263               1.674269
307               1.016308               0.118629
..                     ...                    ...
497              -0.421756               2.121061
830               2.820048              -1.223126
139              -0.312143              -3.046795
131               0.398070              -1.091916
536              -2.255133              -0.984662

[805 rows x 2 columns]

X Test:
     principal component 1  principal component 2
197              -3.005000              -1.212067
186               0.669455              -0.915653
393              -0.543672               0.332344
340              -0.461581               3.139182
991               0.427644              -0.418279
..                     ...                    ...
717    

In [88]:
# Create linear SVM model
# model = MultiOutputClassifier(svm.SVC(kernel='linear'))
model = svm.SVC(kernel='linear')

# Train the model
model.fit(x_train, y_train)

# Make predictions
predictions = model.predict(x_test)

# Measure accuracy
acc = accuracy_score(y_test, predictions)

# Print predictions, actual, and accuracy score
print("Predictions:", predictions)
print("Actual:", y_test)
print("Accuracy:", acc)

# Show confusion matrix
# print(multilabel_confusion_matrix(y_test, predictions))
print(confusion_matrix(y_test, predictions))

Predictions: ['robust' 'robust' 'robust' 'prefrail_mci' 'robust' 'robust' 'robust'
 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'robust' 'robust' 'prefrail_mci' 'prefrail_mci' 'robust' 'robust'
 'robust' 'robust' 'prefrail_mci' 'robust' 'robust' 'prefrail_mci'
 'prefrail_mci' 'prefrail_mci' 'robust' 'robust' 'robust' 'robust'
 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'prefrail_mci' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'robust' 'robust' 'robust' 'prefrail_mci' 'prefrail_mci' 'robust'
 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'prefrail_mci' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'prefrail_mci' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'robust' 'robust' 'robust' 'robust' 'prefrail_mci' 'robust' 'robust'
 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust' 'robust'
 'prefrail_mci' 'prefrail_mci' 'robust' 'robust' 'robust' 'robust'
 'prefrail